<a href="https://colab.research.google.com/github/Yoonsun222/CapstoneDesign/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df1  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.01.31-02.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df2  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.07-02.13.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df3  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.14-02.20.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df4 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.21-02.27.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df5 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.28-03.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
weather_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/기상청데이터.csv", encoding = 'euc-kr', parse_dates=['등록일자'])

df = pd.concat([df1,df2,df3,df4,df5],axis=0, join='inner')



In [59]:
df['등록일자'] = df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

df

,시리얼,관측시각,온도(℃),습도(%),미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자
0,V10G1802191,2.021050e+11,20,49,5,4,99,405,2022-01-31 00:00
1,V10G1802099,2.021050e+11,21,46,4,3,106,447,2022-01-31 00:00
2,V10G1802225,2.021050e+11,21,49,4,3,27,461,2022-01-31 00:00
3,V10G1802415,2.021050e+11,17,49,4,3,581,383,2022-01-31 00:00
4,V10G1802677,2.021050e+11,21,46,5,4,7,570,2022-01-31 00:00
...,...,...,...,...,...,...,...,...,...
642691,V10G1803626,2.021050e+11,22,49,3,2,143,703,2022-03-06 23:00
642692,V10G1802586,2.021050e+11,21,44,3,3,10,372,2022-03-06 23:00
642693,V10G1802051,2.021050e+11,21,49,4,3,25,69,2022-03-06 23:00
642694,V10G1802859,2.021050e+11,21,61,3,3,4,383,2022-03-06 23:00


In [60]:
weather_df['등록일자'] = weather_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

weather_df

,등록일자,기온(°C),풍속(m/s),습도(%),일조(hr),전운량(10분위)
0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
1,2022-01-31 02:00,-5.0,2.3,52,0.0,0
2,2022-01-31 03:00,-5.3,1.6,55,0.0,0
3,2022-01-31 04:00,-5.8,1.0,55,0.0,0
4,2022-01-31 05:00,-6.1,1.9,58,0.0,0
...,...,...,...,...,...,...
811,2022-03-05 20:00,1.5,3.9,37,0.0,0
812,2022-03-05 21:00,0.7,3.3,40,0.0,0
813,2022-03-05 22:00,0.0,2.9,40,0.0,0
814,2022-03-05 23:00,-0.3,3.0,41,0.0,0


In [61]:
data = pd.merge(left = df , right = weather_df, how = "inner", on = "등록일자")
data.head(100)
data = data.drop_duplicates(['시리얼','등록일자'], keep='first')

#temp = df[df['시리얼']== 'V01T1622211']

In [78]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data["시리얼"] = encoder.fit_transform(data["시리얼"])

data = data.reset_index()


In [79]:
total_data = data[data.columns[2:]]
total_data = total_data.replace(-9999,np.nan)
total_data = total_data.fillna(method='bfill')
total_data = total_data.fillna(method='pad')
total_data



,관측시각,온도(℃),습도(%)_x,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자,기온(°C),풍속(m/s),습도(%)_y,일조(hr),전운량(10분위)
0,2.022010e+11,0,41,65,24,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
1,2.022010e+11,1,40,1,1,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
2,2.022010e+11,0,42,38,14,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
3,2.022010e+11,0,39,1,2,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
4,2.022010e+11,0,41,30,11,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283709,2.022030e+11,3,31,27,9,129.0,318.0,2022-03-06 00:00,-0.6,3.4,43,0.0,0
283710,2.022030e+11,3,31,1,2,129.0,318.0,2022-03-06 00:00,-0.6,3.4,43,0.0,0
283711,2.022030e+11,21,15,23,17,129.0,318.0,2022-03-06 00:00,-0.6,3.4,43,0.0,0
283712,2.022030e+11,2,33,1,2,129.0,318.0,2022-03-06 00:00,-0.6,3.4,43,0.0,0


In [84]:
X =  total_data[total_data.columns[[1,3,4,5,6,8,9,10,11]]]
Y = total_data[total_data.columns[[2]]]

In [86]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=22)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)


X_train_scaled

,온도(℃),미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),기온(°C),풍속(m/s),습도(%)_y,일조(hr)
165458,0.945312,0.618429,0.623206,0.001955,0.1684,0.328,0.128205,0.394366,0.0
226762,0.835938,0.618429,0.623830,0.001173,0.1528,0.656,0.384615,0.845070,0.0
13870,0.937500,0.619666,0.625078,0.000704,0.1740,0.436,0.500000,0.549296,0.4
126451,0.953125,0.619666,0.625078,0.001173,0.1720,0.564,0.435897,0.943662,0.0
157793,0.960938,0.620284,0.625702,0.002581,0.1564,0.224,0.205128,0.408451,1.0
...,...,...,...,...,...,...,...,...,...
219638,0.945312,0.620903,0.625702,0.000547,0.2280,0.712,0.525641,0.549296,0.2
120166,0.953125,0.620903,0.625702,0.001095,0.0832,0.524,0.141026,0.535211,0.0
24932,0.921875,1.000000,1.000000,0.008211,0.1788,0.296,0.166667,0.521127,0.0
162752,0.945312,0.620284,0.625078,0.008290,0.1788,0.416,0.294872,0.253521,0.0


In [ ]:
from sklearn.svm import SVC

model=SVC()
model.fit(X_train_scaled, Y_train.values.ravel())
pred_train=model.predict(X_train_scaled)
model.score(X_train_scaled, Y_train)
pred_test=model.predict(X_test_scaled)
model.score(X_test_scaled, Y_test)